# Counting shapes

Training times for this task are too long to do interactively, so they were done in batch using a Python scrip [`count_shapes.py`](count_shapes.py) and a PBS script [`count_shapes.pbs`](count_shapes.pbs).  Here, we analyse the results of that computation.

## Required modules

In [17]:
from data_utils import process_data
from history_utils import plot_history
from keras.models import load_model
import numpy as np
import pickle

## History

The training history containing the loss and accuracty informatino as a function of the training epochs has been saved as a pickle file.

In [18]:
with open('count_shapes_hist.pkl', 'rb') as hist_file:
    history = pickle.load(hist_file)

EOFError: Ran out of input

In [ ]:
plot_history(history)

## Model

The model itself has been saved as an HDF5 file

In [19]:
model = load_model('count_shapes.h5')

We can load a small test set to evaluate performance and analyse network properties.

In [20]:
x_test, y_test = process_data('test_multi_obj_data.h5')

In [ ]:
model.evaluate(x_test, y_test)

 6784/20000 [=========>....................] - ETA: 1:38